### This is also a little project of selenium and beautifulsoup. One of my telegram client wanted to scrapp data from a betting website. So, I made it for him.

In [1]:
import pandas as pd 
import numpy as np 
import re 
import warnings
warnings.filterwarnings('ignore')
import requests
from bs4 import BeautifulSoup

In [6]:
from selenium import webdriver
import time
from selenium.webdriver.common.by import By



In [35]:
driver=webdriver.Firefox()

In [4]:
url='https://football.nowgoal8.com/league/2014-2015/36'

In [36]:
driver.get(url)

In [7]:
column_mapping = {'New_Column':'Company',
    'a': 'Home',
    'b': 'Asian Handicap',
    'c': 'Away',
    'd': 'Home',
    'e': 'Draw',
    'f': 'Away',
    'g': 'Over',
    'h': 'Goals',
    'i': 'Under'
}

In [8]:
# a_element.click()
dataframes=[]


In [37]:
try:
    # Find the list of <td> elements with the specified class attribute
    
    td_elements = driver.find_elements(By.CLASS_NAME, "lsm2")
    for i, td_element in enumerate(td_elements[1:3], start=1):
        # Click on the element
        td_element.click()

        time.sleep(3)
        a_element = driver.find_elements(By.CLASS_NAME,"analyze-icon")
       
        for inner_index, inner_element in enumerate(a_element):
        # Open a new tab by clicking the inner element
            inner_element.click()

            time.sleep(5)


            new_tab_window = driver.window_handles[-1]
            driver.switch_to.window(new_tab_window)
            # window_handles = driver.window_handles
            # new_tab_handle = window_handles[1]
            # driver.switch_to.window(new_tab_handle)


            time.sleep(20)


            odds=driver.find_element(By.ID,'chooseOddsType')
            odds.click()

            odds_d=driver.find_element(By.ID,'oddsType_4')
            odds_d.click()     

            html_source = driver.page_source
            soup = BeautifulSoup(html_source, 'lxml')

        
            team=soup.find_all('span',class_='sclassName')
            home_1=team[0]
            away_1=team[1] 
            T_home=home_1.get_text(strip=True)
            T_away=away_1.get_text(strip=True)
            s=soup.find_all('div',class_='score')
            s_1=s[0]
            s1=s_1.get_text(strip=True)
            s_2=s[1]
            s2=s_2.get_text(strip=True)
            score=[s1+'-'+s2]
            r=soup.find('span',title='Score 1st Half').text
            table=soup.find('table',class_='team-table-other')
            table_1=str(table)
            soup_1=BeautifulSoup(table_1,'html.parser')
            td_elements = soup_1.find_all("td", {"data-o": True})  # Find all <td> elements with data-o attribute

            collected_data = []

            for td in td_elements:
                collected_data.append(td.get_text(strip=True))  # Collect the text content of the <td> elements

            # print("Collected Data:", collected_data)
            grouped_dict={}
            for i in range(0, len(collected_data), 9):
                group_key = f'Group {i // 9 + 1}'
                group = collected_data[i:i + 9]
                grouped_dict[group_key] = group

                # Print the grouped dictionary
            # for key, value in grouped_dict.items():
            #     print(f"{key}: {value}")

            desired_groups = ['Group 1', 'Group 2', 'Group 3', 'Group 4', 'Group 9', 'Group 10', ]

            # Initialize an empty dictionary to store the selected groups
            selected_groups_dict = {}

            # Copy the desired groups to the new dictionary
            for group_key in desired_groups:
                if group_key in grouped_dict:
                    selected_groups_dict[group_key] = grouped_dict[group_key]

            # Print the selected groups dictionary
            # for key, value in selected_groups_dict.items():
            #     print(f"{key}: {value}")

            df = pd.DataFrame.from_dict(selected_groups_dict, orient='index')

            # Assign column names starting from 'a' to 'z'
            df.columns = list('abcdefghijklmnopqrstuvwxyz')[:len(df.columns)]

            new_column_values = ['bet365'] * 2 + ['sbobet'] * 2 + ['188 bet'] * 2

            # Insert the new column at the beginning of the DataFrame
            df.insert(0, 'New_Column', new_column_values)
            
            df.rename(columns=column_mapping,inplace=True)
            df=df.reset_index(drop=True)
            df['Year']='2014-2015'
            df['Score']=score[0]
            df['H-T']=r
            df['T_Home']=T_home
            df['T_Away']=T_away


            pattern = ['first', 'live']

            # Calculate the total number of rows in the DataFrame
            total_rows = len(df)

            # Repeat the pattern to match the total number of rows
            type_values = (pattern * (total_rows // len(pattern))) + pattern[:total_rows % len(pattern)]

            # Add the 'type' column to the DataFrame
            df['type'] = type_values

           
            dataframes.append(df)

            # Close the new tab
            driver.close()

            # Switch back to the main page
            driver.switch_to.window(driver.window_handles[0])



except Exception as e:
    print("An exception occurred:", e)
        
finally:
    print('done')

done


In [56]:
combined_dataframe = pd.concat(remaining_dataframes, ignore_index=True)

In [59]:
pattern_length = 60
total_rows = len(combined_dataframe)
repeating_pattern = list(range(1, (total_rows // pattern_length) + 1)) * pattern_length
combined_dataframe['Page_No'] = repeating_pattern

# If the total number of rows is not a multiple of pattern_length
if total_rows % pattern_length != 0:
    remaining = total_rows % pattern_length
    combined_dataframe['Page_No'] = combined_dataframe['Page_No'].tolist() + [repeating_pattern[-1]] * remaining

In [22]:
combined_dataframe.to_csv('Sample.csv',index=False)

In [63]:
block_size = 60
combined_dataframe['Page_No'] = (combined_dataframe.index // block_size) + 1

In [66]:
combined_dataframe

,Company,Home,Asian Handicap,Away,Home,Draw,Away,Over,Goals,Under,Year,Score,H-T,T_Home,T_Away,type,Page_No
0,bet365,2.05,0/0.5,1.85,2.30,3.40,3.00,2.05,2.5,1.85,2014-2015,0-0,0-0,Aston Villa,Newcastle United,first,1
1,bet365,1.85,0,2.075,2.50,3.40,3.00,1.95,2/2.5,1.95,2014-2015,0-0,0-0,Aston Villa,Newcastle United,live,1
2,sbobet,2.14,0/0.5,1.80,2.52,3.10,2.85,2.07,2.5,1.83,2014-2015,0-0,0-0,Aston Villa,Newcastle United,first,1
3,sbobet,1.87,0,2.08,2.62,3.10,2.91,1.96,2/2.5,1.96,2014-2015,0-0,0-0,Aston Villa,Newcastle United,live,1
4,188 bet,1.82,0,2.12,2.40,3.20,2.70,2.05,2.5,1.85,2014-2015,0-0,0-0,Aston Villa,Newcastle United,first,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
115,bet365,1.925,-0.5,1.975,4.50,3.75,1.85,1.90,2.5,2.00,2014-2015,1-1,1-1,Leicester City,Arsenal,live,2
116,sbobet,2.07,-0.5,1.83,3.95,3.35,1.82,1.98,2.5/3,1.92,2014-2015,1-1,1-1,Leicester City,Arsenal,first,2
117,sbobet,1.98,-0.5,1.97,3.95,3.65,1.92,1.92,2.5,2.00,2014-2015,1-1,1-1,Leicester City,Arsenal,live,2
118,188 bet,1.81,0/0.5,2.09,3.95,3.45,1.80,2.00,2.5/3,1.90,2014-2015,1-1,1-1,Leicester City,Arsenal,first,2


In [54]:
remaining_dataframes = dataframes[10:]

20

In [25]:
combined_dataframe['Score'] = "'" + combined_dataframe['Score'].astype(str)
combined_dataframe['H-T'] = "'" + combined_dataframe['H-T'].astype(str)
combined_dataframe.to_csv('output.csv', index=False)

In [32]:
soup1=BeautifulSoup(html_source,'html.parser')

In [33]:
print(soup.prettify())

<html lang="en-my" xml:lang="en-my" xmlns="http://www.w3.org/1999/xhtml">
 <head>
  <meta content="webkit" name="renderer"/>
  <meta content="webkit" name="force-rendering"/>
  <meta content="IE=edge,chrome=1" http-equiv="X-UA-Compatible"/>
  <meta content="width=1220, user-scalable=yes" name="viewport"/>
  <meta content="text/html; charset=utf-8" http-equiv="Content-Type"/>
  <title>
   Burnley VS Chelsea - Football Analysis, Preview, Head to Head Stats, Prediksi
  </title>
  <meta content="Burnley, Chelsea, live score, football analysis, head to head, standing, fixture, prediksi bola" name="keywords"/>
  <meta content="Check out the most complete match analysis for Burnley VS Chelsea of English Premier League on 19-08-2014. Head to head statistics, live odds comparison, historical record, injured &amp; suspension and more." name="description"/>
  <link href="https://www.nowgoal8.com" hreflang="en-my" rel="alternate"/>
  <meta content="Burnley VS Chelsea - Football Analysis, Preview, 